<a href="https://colab.research.google.com/github/phpaivamotta/pdf-headers/blob/main/PDF_Headers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyMuPDF
!pip install roman

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 22.4 MB/s eta 0:00:00


In [2]:
import fitz
import roman

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
font = fitz.Font(fontname="BA", fontfile="/content/drive/MyDrive/PDF Headers/Book Antiqua Font Files/BKANT.TTF")

## **Inputs**

In [17]:
# Update these paths based on where the files are located in your Google Drive
input_pdf = '/content/drive/MyDrive/PDF Headers/PDF wo Headers/test MAS-233266-FPC-1016-501_R1 REPORT ONLY.pdf'
output_pdf = '/content/drive/MyDrive/PDF Headers/PDF w Headers/MAS-233266-FPC-1016-501_R1 REPORT ONLY - w Headers.pdf'

# Header text
company_name = 'Formosa Plastics'
project_name = 'IEM C-602A Column and Caustic Piping'
document_description = 'Detailed Pipe Stess Analysis, Revision 1'
date = 'March 21, 2024'
maverick = 'Maverick Applied Science, Inc.'
document_name = 'MAS-233266-FPC-1016-501'
attachment_name = 'Attachment A'
footer_attachment = 'A - '

# Margins (margins seen below are in REMs, not inches)
LEFT_MARGIN = 61
TOP_MARGIN_1 = 45
TOP_MARGIN_2 = 60
TOP_MARGIN_3 = 75
BOTTOM_MARGIN = 35

# Ranges
# Main header (on top-right and left of page)
main_header_start = 2
main_header_finish = 30
# Center header (on top-center of page)
center_header_start = 5
center_header_finish = 30
# Footer report roman
footer_roman_start = 2
footer_roman_finish = 5
# Footer report normal
footer_start = 6
footer_finish = 25
# Footer attachment
footer_attachment_start = 26
footer_attachment_finish = 30

# Create ranges
main_header_range = range(main_header_start, main_header_finish + 1)
center_header_range = range(center_header_start, center_header_finish + 1)
footer_roman_range = range(footer_roman_start, footer_roman_finish + 1)
footer_range = range(footer_start, footer_finish + 1)
footer_attachment_range = range(footer_attachment_start, footer_attachment_finish + 1)

In [19]:
def add_header_to_pdf(
    input_pdf, output_pdf, company_name, project_name, document_description, date, maverick, document_name, attachment_name, footer_attachment,
    main_header_range, center_header_range, footer_range, footer_roman_range, footer_attachment_range
    ):

    # Open input PDF file
    doc = fitz.open(input_pdf)

    # Get width and height of first page, which should be the same as all other pages
    page_width = doc[0].rect.width
    page_height = doc[0].rect.height

    # Get length of text to be written on right side of the page
    date_length = font.text_length(date, fontsize=12)
    maverick_length = font.text_length(maverick, fontsize=12)
    doc_name_length = font.text_length(document_name, fontsize=12)

    # Get length of attachment text to be written on center of page
    attachment_name_length = font.text_length(attachment_name, fontsize=12)

    for i, page in enumerate(doc, start=1):
        # Load Book Antiqua font file
        page.insert_font(fontfile="/content/drive/MyDrive/PDF Headers/Book Antiqua Font Files/BKANT.TTF", fontname="BA")

        # Write headers to page
        if i in main_header_range:
          # Headers on left of page
          page.insert_text((LEFT_MARGIN, TOP_MARGIN_1), company_name, fontsize=12, fontname="BA", color=(0,0,0))
          page.insert_text((LEFT_MARGIN, TOP_MARGIN_2), project_name, fontsize=12, fontname="BA", color=(0,0,0))
          page.insert_text((LEFT_MARGIN, TOP_MARGIN_3), document_description, fontsize=12, fontname="BA", color=(0,0,0))
          # Headers on right of page
          page.insert_text((page_width - (LEFT_MARGIN + date_length), TOP_MARGIN_1), date, fontsize=12, fontname="BA", color=(0,0,0))
          page.insert_text((page_width - (LEFT_MARGIN + maverick_length), TOP_MARGIN_2), maverick, fontsize=12, fontname="BA", color=(0,0,0))
          page.insert_text((page_width - (LEFT_MARGIN + doc_name_length), TOP_MARGIN_3), document_name, fontsize=12, fontname="BA", color=(0,0,0))

        # Headers/footers in center of page
        # Attachment name header
        if i in center_header_range:
          page.insert_text(((page_width/2) - (attachment_name_length/2), TOP_MARGIN_1), attachment_name, fontsize=12, fontname="BA", color=(0,0,0))
        # Report roman numeral footer
        if i in footer_roman_range:
          roman_num = roman.toRoman((i + 1) - footer_roman_start)
          footer_roman_length = font.text_length(roman_num, fontsize=12)
          page.insert_text(((page_width/2) - (footer_roman_length/2), (page_height - BOTTOM_MARGIN)), roman_num, fontsize=12, fontname="BA", color=(0,0,0))
        # Report footer
        if i in footer_range:
          page_num = str((i + 1) - footer_start)
          footer_length = font.text_length(page_num, fontsize=12)
          page.insert_text(((page_width/2) - (footer_length/2), (page_height - BOTTOM_MARGIN)), page_num, fontsize=12, fontname="BA", color=(0,0,0))
        # Footer attachment range
        if i in footer_attachment_range:
          attachment_num = str((i + 1) - footer_attachment_start)
          footer_attachment_length = font.text_length((footer_attachment + attachment_num), fontsize=12)
          page.insert_text(((page_width/2) - (footer_length/2), (page_height - BOTTOM_MARGIN)), f'{footer_attachment}{(i + 1) - footer_attachment_start}', fontsize=12, fontname="BA", color=(0,0,0))

    doc.save(output_pdf)
    print(f"Header added. File saved as '{output_pdf}'")

In [20]:
add_header_to_pdf(
    input_pdf, output_pdf, company_name, project_name, document_description,
    date, maverick, document_name, attachment_name, footer_attachment, main_header_range,
    center_header_range, footer_range, footer_roman_range, footer_attachment_range
    )

Header added. File saved as '/content/drive/MyDrive/PDF Headers/PDF w Headers/MAS-233266-FPC-1016-501_R1 REPORT ONLY - w Headers.pdf'
